# Ejercicios con Dataset Superstore

In [ ]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Usar el método read_csv de Pandas (pd) que recibe como argumento el archivo csv y retorna un dataframe
import os
os.chdir('/content/drive/MyDrive/datasets')
df = pd.read_csv("superstore.csv", encoding="latin1")

In [ ]:
# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

In [ ]:
# Tomemos algunas muestras
df.sample()

In [ ]:
df["city"].unique()

In [ ]:
# Limpieza básica y conversión de tipo de datos
df = df.dropna(subset=["order_date", "sales", "profit"])
# Convertimos el campo fecha de string a datetime
df["order_date"] = pd.to_datetime(df["order_date"])

In [ ]:
# Transformación: generar las columnas año y mes a partir del campo date
df["year"]=df["order_date"].dt.year

## Analizamos las ventas totales por ciudades

In [ ]:
# Hagamos un grafico de barras de las mejores 10 ventas acumuladas por ciudad
df_aggr=df.groupby("city").agg(
    {"sales":"sum",
    "profit":"sum"}
    ).sort_values(by="sales", ascending=False).head(10)
df_aggr=df_aggr.reset_index().rename(columns={"sales":"Ventas", "city":"Ciudades", "profit":"Ganancias"})
df_aggr

In [ ]:
# Estilo y paleta
sns.set_theme(               # set_theme combina estilo + contexto + paleta
    style="whitegrid",       # opciones: 'white', 'whitegrid', 'dark', 'darkgrid', 'ticks'
    context="notebook",          # escala general: 'paper', 'notebook', 'talk', 'poster'
    palette="Set3"           # paleta de colores base: "deep", "muted", "pastel" / Set1, Set2, Set3
)

# Ajustamos el lienzo
plt.figure(figsize=(10,6))

sns.barplot(data=df_aggr, x="Ciudades", y="Ventas", width=0.8, label="Acumulados por Ciudades")

# Ticks (valores de eje)
plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', rotation=45, labelsize=10)

# Ajustes de título y ejes
plt.title('Ventas acumuladas por ciudades', fontsize=14)
plt.xlabel('Ciudades')
plt.ylabel('Ventas')
plt.legend()
plt.tight_layout()


In [ ]:
# Y representemos en un grafico de barras apilado la relacion ventas/ganancias
plt.figure(figsize=(16,6))
# Primer nivel (abajo)
plt.bar(df_aggr["Ciudades"], df_aggr["Ventas"], label="Ventas")

df_aggr['color'] = df_aggr['Ganancias'].apply(lambda x: 'green' if x >= 0 else 'red')
# Segundo nivel (arriba)
plt.bar(df_aggr["Ciudades"], df_aggr["Ganancias"],color=df_aggr['color'],
        bottom=df_aggr["Ventas"],
        label="Ganancias")

In [ ]:
# Formato largo
df_long = df_aggr.melt(
    id_vars="Ciudades",
    value_vars=["Ventas", "Ganancias"],
    var_name="Tipo",
    value_name="Valor"
)
df_long.head()

In [ ]:
# Definimos el lienzo
plt.figure(figsize=(10,6))

sns.barplot(
    data=df_long,
    x="Ciudades",
    y="Valor",
    hue="Tipo",
    dodge=True,           # ← SIDE BY SIDE
)

# Ticks (valores de eje)
plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', rotation=45, labelsize=10)

# Ajustes de títulos y ejes
plt.title("Ventas vs Ganancias (Side-by-Side)")
plt.xlabel("Ciudades")
plt.ylabel("Monto")
plt.tight_layout()
plt.show()


## Analizamos las ventas totales por ciudades, comparando por años

In [ ]:
# Buscamos las 3 ciudades con mayor volumen de ventas
df_aggr=df.groupby(["city"], as_index=False).agg(
    {"sales":"sum"}
    ).sort_values(by=["sales"], ascending=False).head(3)
df_aggr

In [ ]:
# Filtramos del df original estas tres ciudades
df_filtrado=df[df["city"].isin(df_aggr["city"])]


In [ ]:
# Hagamos un grafico de barras de las mejores 10 ventas acumuladas por ciudad
df_aggr1=df_filtrado.groupby(["city", "year"]).agg(
    {"sales":"sum"}
    )
df_aggr1=df_aggr1.reset_index().rename(columns={"sales":"Ventas", "city":"Ciudades", "year": "Año"})
df_aggr1

In [ ]:
# Estilo y paleta
sns.set_theme(               # set_theme combina estilo + contexto + paleta
    style="whitegrid",       # opciones: 'white', 'whitegrid', 'dark', 'darkgrid', 'ticks'
    context="notebook",          # escala general: 'paper', 'notebook', 'talk', 'poster'
    palette="Set3"           # paleta de colores base: "deep", "muted", "pastel" / Set1, Set2, Set3
)

# Ajustamos el lienzo
plt.figure(figsize=(10,6))

sns.barplot(
    data=df_aggr1,
    x="Ciudades",
    y="Ventas",
    hue="Año",
    dodge=True,           # ← SIDE BY SIDE
    palette="Set3"
)

# Ticks (valores de eje)
plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', labelsize=10)

# Ajustes de títulos y ejes
plt.title("Ventas vs Ganancias (Side-by-Side)")
plt.xlabel("Ciudades")
plt.ylabel("Monto")
plt.tight_layout()
plt.show()

# Bonus track

El Rose Chart o Coxcomb Chart es un tipo de gráfico polar creado por Florence Nightingale en 1858 para mostrar de manera impactante las causas de muerte de los soldados británicos durante la Guerra de Crimea.

⭐ Características principales

* Es un gráfico circular dividido en segmentos iguales (como pétalos).
* Cada segmento representa un mes.
* La altura del pétalo (radio) representa la magnitud del valor.

Nightingale lo usó para mostrar tres categorías de muertes:

* Enfermedades prevenibles
* Heridas en combate
* Otras causas

Los colores diferenciaban cada causa.

Su forma visual enfatiza cómo las enfermedades mataban mucho más que las batallas.

🎯 Objetivo del gráfico

Demostrar de forma clara y contundente que la mayoría de las muertes se debían a mala higiene y enfermedades, no al combate.
Este gráfico fue clave para convencer al gobierno británico de implementar reformas sanitarias en los hospitales militares.

🔥 Por qué es famoso

* Es uno de los primeros ejemplos de storytelling con datos.
* Pionero en data visualization.
* Impactó políticas públicas.

🧠 Concepto técnico

* No es un pie chart.
* No representa proporciones.
* Representa valores absolutos en un espacio polar.
* Se llama coxcomb chart o polar area chart.

[Florence Nightingale's Famous Rose Chart](https://www.youtube.com/watch?v=JZh8tUy_bnM)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def nightingale_rose(df, col_categoria="Ciudades", col_valor="Ventas",
                     title="Nightingale Rose Diagram"):
    """
    df: DataFrame ordenado con dos columnas
    col_categoria: nombre de la columna de categorías
    col_valor: nombre de la columna numérica
    """

    categorias = df[col_categoria].tolist()
    valores = df[col_valor].tolist()

    N = len(df)                           # número de filas del df
    angles = np.linspace(0, 2*np.pi, N, endpoint=False)

    # Paleta automática (tab20 soporta hasta 20 colores, tab20b + tab20c permiten más)
    if N <= 15:
        colors = plt.cm.tab20(np.linspace(0, 1, N))
    else:
        # Para más de 20 categorías, usamos una paleta continua
        colors = plt.cm.viridis(np.linspace(0, 1, N))

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(111, polar=True)

    # Barras
    ax.bar(
        angles,
        valores,
        width=2 * np.pi / N,
        bottom=0,
        color=colors,
        edgecolor="white",
        linewidth=1
    )

    # Etiquetas
    for angle, value, label in zip(angles, valores, categorias):
        ax.text(
            angle,
            value + (max(valores) * 0.05),
            f"{label}\n{value}",
            ha="center",
            va="center",
            fontsize=8
        )

    # Limpieza estética
    ax.set_yticklabels([])
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.title(title, fontsize=16, pad=20)
    plt.show()



In [ ]:
# Hagamos un grafico de barras de las mejores 10 ventas acumuladas por ciudad
df_aggr=df.groupby("city").agg(
    {"sales":"sum"}
    ).sort_values(by="sales", ascending=False).head(10)
df_aggr=df_aggr.reset_index().rename(columns={"sales":"Ventas", "city":"Ciudades"})
df_aggr

In [ ]:
# Graficamos con Nightingale Rose
nightingale_rose(df_aggr, "Ciudades", "Ventas",
                 title="Nightingale Rose Diagram – 10 Ciudades")
